<a href="https://colab.research.google.com/github/sayanbanerjee32/keyword_search/blob/main/Semantic_search_BERT_FAISS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Download data

In [1]:
%%capture
# To import kaggle datasets
!pip install --upgrade pip 
!pip install kaggle==1.5.6
!pip install datasets
!pip install transformers 
!pip install faiss-gpu

In [2]:
# # import colab libraries
# from google.colab import files
# # import kaggle json to connect to kaggle user account to download datsets
# files.upload()

# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# The Kaggle API client expects this file to be in ~/.kaggle,
# so lets move it there.
!mkdir ~/.kaggle
!cp /content/drive/MyDrive/Kaggle_api_auth/kaggle.json ~/.kaggle/

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [4]:
# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
# download our dataset
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews -p 'datasets'

imdb-dataset-of-50k-movie-reviews.zip: Skipping, found more recently modified local copy (use --force to force download)


In [6]:
#unzip
!unzip datasets/imdb-dataset-of-50k-movie-reviews.zip -d datasets/

Archive:  datasets/imdb-dataset-of-50k-movie-reviews.zip
replace datasets/IMDB Dataset.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


## Sentence BERT
https://huggingface.co/course/chapter5/6?fw=tf

In [7]:
import re
def fix_text(txt):
    return re.sub(r'(<br />)+',' ',txt)
    # return clean(txt,
    #             fix_unicode=True,               # fix various unicode errors
    #             to_ascii=True,                  # transliterate to closest ASCII representation
    #             lower=True,                     # lowercase text
None    #             no_line_breaks=True,           # fully strip line breaks as opposed to only normalizing them
    #             no_urls=False,                  # replace all URLs with a special token
    #             no_emails=True,                # replace all email addresses with a special token
    #             no_phone_numbers=True,         # replace all phone numbers with a special token
    #             no_numbers=True,               # replace all numbers with a special token
    #             no_digits=False,                # replace all digits with a special token
    #             no_currency_symbols=True,      # replace all currency symbols with a special token
    #             no_punct=True,                 # remove punctuations
    #             replace_with_punct="<PUNCT>",          # instead of removing punctuations you may replace them
    #             replace_with_url="<URL>",
    #             replace_with_email="<EMAIL>",
    #             replace_with_phone_number="<PHONE>",
    #             replace_with_number="<NUMBER>",
    #             replace_with_digit="0",
    #             replace_with_currency_symbol="<CUR>",
    #             lang="en"                       # set to 'de' for German special handling
    #         )

In [8]:
import pandas as pd
df = pd.read_csv('datasets/IMDB Dataset.csv')
df.shape


(50000, 2)

In [9]:
df['review'] = df['review'].apply(fix_text)
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. The filming te...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [10]:
from datasets import Dataset

imdb_dataset = Dataset.from_pandas(df.iloc[0:1000,:])
imdb_dataset

Dataset({
    features: ['review', 'sentiment'],
    num_rows: 1000
})

## Create word embedding

In [11]:
from transformers import AutoTokenizer, TFAutoModel

model_ckpt = "sentence-transformers/multi-qa-mpnet-base-dot-v1"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = TFAutoModel.from_pretrained(model_ckpt, from_pt=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFMPNetModel: ['embeddings.position_ids']
- This IS expected if you are initializing TFMPNetModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFMPNetModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFMPNetModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMPNetModel for predictions without further training.


In [12]:
def cls_pooling(model_output):
    return model_output.last_hidden_state[:, 0]

def get_embeddings(text_list):
    encoded_input = tokenizer(
        text_list, padding=True, truncation=True, return_tensors="tf"
    )
    encoded_input = {k: v for k, v in encoded_input.items()}
    model_output = model(**encoded_input)
    return cls_pooling(model_output)

In [13]:
embedding = get_embeddings(imdb_dataset["review"][0])
embedding.shape

TensorShape([1, 768])

In [14]:
embeddings_dataset = imdb_dataset.map(
    lambda x: {"embeddings": get_embeddings(x["review"]).numpy()[0]}
)

INFO:tensorflow:Assets written to: ram://33ba178c-d92f-467b-80b4-99abebd2fbdd/assets


INFO:tensorflow:Assets written to: ram://33ba178c-d92f-467b-80b4-99abebd2fbdd/assets


0ex [00:00, ?ex/s]

### FAISS for similarity search

In [15]:
embeddings_dataset.add_faiss_index(column="embeddings")

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset({
    features: ['review', 'sentiment', 'embeddings'],
    num_rows: 1000
})

In [20]:
question = "game with awkward hand position"
question_embedding = get_embeddings([question]).numpy()
question_embedding.shape

(1, 768)

In [21]:
scores, samples = embeddings_dataset.get_nearest_examples(
    "embeddings", question_embedding, k=10
)

In [22]:
samples_df = pd.DataFrame.from_dict(samples)
samples_df["scores"] = scores
samples_df.sort_values("scores", ascending=False, inplace=True)

In [23]:
for _, row in samples_df.iterrows():
    print(f"Review: {row.review}")
    print("=" * 50)
    print()

Review: This movie is so God-awful that it was literally a chore to watch. I wanted to eject it from my vcr and throw it across the room, but kept thinking (foolishly) that it would eventually get funny and then everything would be all right. "You lose, we win, yay!" This movie should be required viewing for anyone who even once entertained the thought that Jackie Mason was funny. After that, beat them ove the head with this movie until the tape cracks. And if you're even considering renting this turd (or worse yet, have!) I have one thing to ask of you: didn't you even look at the cover? I mean, with crap like this you can tell with just a glance how bad it is! "Oy vey!" This movie sucked.

Review: Steven Seagal's films of late have not exactly been good, but this is by far the worst since The Patriot. The plot makes no sense what so ever; it is never clear in what the relationships between the characters are, who works for who or who is double crossing who. The film is completely dis